<a href="https://colab.research.google.com/github/rohinishimpatwar/The-NLP-News-Sentiment-Trading-Strategy/blob/master/NLP/Part3_Sentiment_Model_BERT_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# import tensorflow as tf
# # Get the GPU device name.
# device_name = tf.test.gpu_device_name()

# # The device name should look like the following:
# if device_name == '/device:GPU:0':
#     print('Found GPU at: {}'.format(device_name))
# else:
#     raise SystemError('GPU device not found')


In [34]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [35]:
!pip install transformers

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
# show plots inline
%matplotlib inline
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/pentagon_quants/code/final_dataset/clean_labeled_news_withid.csv', index_col=[0])
df.head(10)

,news_id,text_no_stopwords,label
0,102874,reuters parcel service ups reported higher net...,1.0
1,63428,sherwin company nyse shw issue press release a...,0.0
2,67414,general electric spinoff public offering ge tr...,1.0
3,113982,ap motorists paying cents gallon trouble under...,1.0
4,121971,intel tumbles make chips semiconductors manufa...,-1.0
5,2187,show chapters surprising group making comeback...,1.0
6,104871,reuters jnj suffered trial loss lawsuit claimi...,-1.0
7,178860,hours ago tech expensive decade brutal billion...,-1.0
8,292185,updated minutes ago samsung firms samsung elec...,-1.0
9,277764,spire nyse sr announced today priced public of...,1.0


In [38]:

df['label'] = df['label'].astype(int) 



df=df.replace(to_replace =-1, 
                 value =2)

df.head(10)

,news_id,text_no_stopwords,label
0,102874,reuters parcel service ups reported higher net...,1
1,63428,sherwin company nyse shw issue press release a...,0
2,67414,general electric spinoff public offering ge tr...,1
3,113982,ap motorists paying cents gallon trouble under...,1
4,121971,intel tumbles make chips semiconductors manufa...,2
5,2187,show chapters surprising group making comeback...,1
6,104871,reuters jnj suffered trial loss lawsuit claimi...,2
7,178860,hours ago tech expensive decade brutal billion...,2
8,292185,updated minutes ago samsung firms samsung elec...,2
9,277764,spire nyse sr announced today priced public of...,1


In [39]:
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(df.text_no_stopwords, df.label, random_state=777, test_size=0.2)
train_inputs, train_labels

(373    shares enterprise plunged percent ceo prepared...
 331    reuters trade tensions rise states vocal onlin...
 223    pm minutes posts adj earnings share reuters st...
 601    minutes regulator warns deutsche bank fixing t...
 241    pembroke business wire axis capital holdings l...
                              ...                        
 669    pm updated hours ago trade body backs plane ma...
 71     adds analyst commentary reuters economy grew t...
 599    reuters longtime business partner fellow billi...
 571    reuters federal communications commission refe...
 103    adds background reuters amazon set buy startup...
 Name: text_no_stopwords, Length: 640, dtype: object, 373    2
 331    2
 223    2
 601    0
 241    1
       ..
 669    0
 71     2
 599    2
 571    2
 103    1
 Name: label, Length: 640, dtype: int64)

In [0]:
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [0]:
# label_list = ['0', '1', '2']
# examples_train = [InputExample(str(i), t, label=str(train_labels.iloc[i])) for i, t in enumerate(train_inputs)]
# train_features = glue_convert_examples_to_features(examples_train, tokenizer, max_length =64,  label_list = label_list, output_mode='classification')

In [0]:
def data_process(inputs, labels, batch_size=32): # df.series
  label_list = ['0', '1', '2']
  examples = [InputExample(str(i), t, label=str(labels.iloc[i])) for i, t in enumerate(inputs)]
  features = glue_convert_examples_to_features(examples, tokenizer, max_length =64,  label_list = label_list, output_mode='classification')

  all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
  all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
  all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
  all_labels = torch.tensor([f.label for f in features], dtype=torch.long)

  # Create the DataLoader for our training set.
  dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
  sampler = RandomSampler(dataset)
  batch_size = 32
  dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)
  return dataloader

In [0]:
train_dataloader = data_process(train_inputs, train_labels)
valid_dataloader = data_process(valid_inputs, valid_labels)

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [46]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)

# Tell pytorch to run this model on the GPU.
model.cuda()

# optimizer
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )

In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 20

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [48]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_token_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, # b_token_type_ids
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    #print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in valid_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=b_token_type_ids, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

======== Epoch 1 / 20 ========
Training...
  Average training loss: 1.01
  Training epcoh took: 0:00:06
Running Validation...
  Accuracy: 0.55
  Validation took: 0:00:00
======== Epoch 2 / 20 ========
Training...
  Average training loss: 0.86
  Training epcoh took: 0:00:05
Running Validation...
  Accuracy: 0.66
  Validation took: 0:00:00
======== Epoch 3 / 20 ========
Training...
  Average training loss: 0.66
  Training epcoh took: 0:00:05
Running Validation...
  Accuracy: 0.69
  Validation took: 0:00:00
======== Epoch 4 / 20 ========
Training...
  Average training loss: 0.47
  Training epcoh took: 0:00:06
Running Validation...
  Accuracy: 0.65
  Validation took: 0:00:00
======== Epoch 5 / 20 ========
Training...
  Average training loss: 0.32
  Training epcoh took: 0:00:05
Running Validation...
  Accuracy: 0.64
  Validation took: 0:00:00
======== Epoch 6 / 20 ========
Training...
  Average training loss: 0.18
  Training epcoh took: 0:00:05
Running Validation...
  Accuracy: 0.65
  Valid

In [49]:
text='Stock raise highly and the company gets big revenue this quarter.'
def predict(text, model=model):
  inputs_1 = tokenizer.encode_plus(text, max_length=512, add_special_tokens=True, return_tensors='pt')
  pred = model(inputs_1['input_ids'].to(device), token_type_ids=inputs_1['token_type_ids'].to(device), attention_mask=inputs_1['attention_mask'].to(device))[0].argmax().item()
  return pred

predict(text)

1

In [51]:
from sklearn.metrics import accuracy_score, confusion_matrix
from collections import Counter
df['pred'] = df['text_no_stopwords'].apply(predict)
print(accuracy_score(df.label, df.pred))
print(Counter(df.pred))
#cm = confusion_matrix(df.label, df.pred, labels=['positive','neutral','negative'])

0.785
Counter({1: 457, 2: 268, 0: 75})


In [56]:
'''import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
cm = confusion_matrix(df.label, df.pred, labels=['positive','neutral','negative'])
labels=['positive','neutral','negative']
df_cm = pd.DataFrame(cm, index = [i for i in labels],
                  columns = [i for i in labels])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, cmap="Blues")'''

'import seaborn as sn\nimport pandas as pd\nimport matplotlib.pyplot as plt\ncm = confusion_matrix(df.label, df.pred, labels=[\'positive\',\'neutral\',\'negative\'])\nlabels=[\'positive\',\'neutral\',\'negative\']\ndf_cm = pd.DataFrame(cm, index = [i for i in labels],\n                  columns = [i for i in labels])\nplt.figure(figsize = (10,7))\nsn.heatmap(df_cm, annot=True, cmap="Blues")'

In [55]:
df_unlabel = pd.read_csv('/content/drive/My Drive/pentagon_quants/code/final_dataset/clean_unlabel_news_withid.csv', index_col=[0])
df_unlabel

,text_no_stopwords
news_id,
77466,company disastrous fool
155315,brinker hour ago
299381,addresses payment hours ago
162935,bottom walmart hours ago
31426,highs activision hours ago
...,...
187014,globe newswire pharma plc aim vrp nasdaq vrna ...
209107,amsterdam veon nasdaq veon euronext amsterdam ...
195881,nyse tsx ntr saskatoon nutrien nutrien announc...


In [57]:
pred = []
count = 0
for text in df_unlabel.text_no_stopwords:
  if count % 5000 == 0:
    print(f'finish {count}')
  cur_pred = predict(text)
  pred.append(cur_pred)
  count += 1

finish 0
finish 5000
finish 10000
finish 15000
finish 20000
finish 25000
finish 30000
finish 35000


In [58]:
from collections import Counter
Counter(pred)

Counter({0: 3328, 1: 25135, 2: 10685})

In [59]:
df_unlabel['pred'] = pred
df_unlabel

,text_no_stopwords,pred
news_id,,
77466,company disastrous fool,2
155315,brinker hour ago,0
299381,addresses payment hours ago,0
162935,bottom walmart hours ago,0
31426,highs activision hours ago,1
...,...,...
187014,globe newswire pharma plc aim vrp nasdaq vrna ...,1
209107,amsterdam veon nasdaq veon euronext amsterdam ...,1
195881,nyse tsx ntr saskatoon nutrien nutrien announc...,1


Combine predcitons and labeled samples

In [61]:
 pred = df_unlabel.reset_index()
pred = pred[['news_id','pred']]
pred = pred.rename(columns={'pred':'label'})
pred['label'] = pred.label.apply(lambda x: -1 if x==2 else x)
pred

,news_id,label
0,77466,-1
1,155315,0
2,299381,0
3,162935,0
4,31426,1
...,...,...
39143,187014,1
39144,209107,1
39145,195881,1
39146,232999,1


In [62]:
labeled = pd.read_csv('/content/drive/My Drive/pentagon_quants/code/final_dataset/clean_labeled_news_withid.csv')
labeled = labeled[['news_id','label']]
labeled['label'] = labeled.label.apply(lambda x: -1 if x==2 else x)
labeled

,news_id,label
0,102874,1.0
1,63428,0.0
2,67414,1.0
3,113982,1.0
4,121971,-1.0
...,...,...
795,223144,1.0
796,268916,-1.0
797,297180,-1.0
798,106502,0.0


In [63]:
combined_df = pd.concat([pred, labeled])
combined_df

,news_id,label
0,77466,-1.0
1,155315,0.0
2,299381,0.0
3,162935,0.0
4,31426,1.0
...,...,...
795,223144,1.0
796,268916,-1.0
797,297180,-1.0
798,106502,0.0


In [64]:
Counter(combined_df.label)

Counter({-1.0: 10973, 0.0: 3457, 1.0: 25518})

In [0]:
combined_df.to_csv('/content/drive/My Drive/pentagon_quants/code/final_dataset/bert_predicted_result_40ks.csv')

In [0]:
import pandas as pd
combined_df = pd.read_csv('/content/drive/Shared with me/pentagon_quants/code/final_dataset/bert_predicted_result_40k.csv')